### Exploring the racial demographics of oil spill locations

In [1]:
# to read and wrangle data
import pandas as pd

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# bring in idle wells data
wells = pd.read_csv('LAC_wells_onshore2.csv')

In [3]:
# test it
wells.head()

,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,Latitude,Longitude
0,Chanchorena,1,Plugged,OG,Oil & Gas,S1375,Steven A. Sayce,33.593762,-118.023328
1,Lease by San Pedro-Point Fermin Oil & Gas Co.,1,Plugged,DH,Dry Hole,7633,San Pedro-Point Fermin Oil & Gas Co.,33.705338,-118.293968
2,Lease by Lighthouse Oil Co.,1,Idle,OG,Oil & Gas,9707,Lighthouse Oil Co.,33.708696,-118.291480
3,Lease by Los Angeles Harbor Oil Dev. Co.,1,Idle,OG,Oil & Gas,5421,Los Angeles Harbor Oil Dev. Co.,33.716400,-118.302544
4,Sepulveda,1,Plugged,DH,Dry Hole,8166,"Surety Holding Co., Ltd.",33.721802,-118.326088


In [4]:
# convert pandas dataframe to geodataframe
geowells = gpd.GeoDataFrame(wells, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(wells.Longitude, wells.Latitude))


In [5]:
# test it
geowells.head()

,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,Latitude,Longitude,geometry
0,Chanchorena,1,Plugged,OG,Oil & Gas,S1375,Steven A. Sayce,33.593762,-118.023328,POINT (-118.02333 33.59376)
1,Lease by San Pedro-Point Fermin Oil & Gas Co.,1,Plugged,DH,Dry Hole,7633,San Pedro-Point Fermin Oil & Gas Co.,33.705338,-118.293968,POINT (-118.29397 33.70534)
2,Lease by Lighthouse Oil Co.,1,Idle,OG,Oil & Gas,9707,Lighthouse Oil Co.,33.708696,-118.291480,POINT (-118.29148 33.70870)
3,Lease by Los Angeles Harbor Oil Dev. Co.,1,Idle,OG,Oil & Gas,5421,Los Angeles Harbor Oil Dev. Co.,33.716400,-118.302544,POINT (-118.30254 33.71640)
4,Sepulveda,1,Plugged,DH,Dry Hole,8166,"Surety Holding Co., Ltd.",33.721802,-118.326088,POINT (-118.32609 33.72180)


In [6]:
geowells.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
geowells_web_mercator = geowells.to_crs(epsg=3857)

In [8]:
wellsIDLE=geowells_web_mercator.loc[geowells_web_mercator['WellStatus']=='Idle']
wellsIDLE.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3617 entries, 2 to 24232
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   LeaseName   3617 non-null   object  
 1   WellNumber  3617 non-null   object  
 2   WellStatus  3617 non-null   object  
 3   WellType    3617 non-null   object  
 4   WellTypeLa  3617 non-null   object  
 5   OperatorCo  3617 non-null   object  
 6   OperatorNa  3617 non-null   object  
 7   Latitude    3617 non-null   float64 
 8   Longitude   3617 non-null   float64 
 9   geometry    3617 non-null   geometry
dtypes: float64(2), geometry(1), object(7)
memory usage: 310.8+ KB


In [9]:
# testing
wellsIDLE.sample(5)

,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,Latitude,Longitude,geometry
20051,Lease by Oceanic Oil Co.,4,Idle,OG,Oil & Gas,6387,Oceanic Oil Co.,34.065891,-118.247711,POINT (-13163274.998 4037653.049)
8652,Lease by Consolidated Mutual Oil Co.,5,Idle,OG,Oil & Gas,1819,Consolidated Mutual Oil Co.,33.812103,-118.314064,POINT (-13170661.358 4003599.904)
23298,N. L. & F.,20-Jan,Idle,OG,Oil & Gas,C0885,California Resources Production Corporation,34.417526,-118.680817,POINT (-13211488.071 4085004.337)
11507,BP Core Hole,2,Idle,CH,Core Hole,C5640,Chevron U.S.A. Inc.,33.873576,-118.025852,POINT (-13138577.745 4011838.978)
22895,Bryson,7,Idle,OG,Oil & Gas,C1380,Caltico Oil Corp.,34.392876,-118.492409,POINT (-13190514.599 4081678.418)


In [10]:
# now lets bring in the census tracts using the ACS 2019 five yr data
tracts=gpd.read_file('acs2019.geojson')

In [11]:
tracts.sample()

,geoid,name,B02001001,"B02001001, Error",B02001002,"B02001002, Error",B02001003,"B02001003, Error",B02001004,"B02001004, Error",...,"B02001006, Error",B02001007,"B02001007, Error",B02001008,"B02001008, Error",B02001009,"B02001009, Error",B02001010,"B02001010, Error",geometry
33,14000US06037104810,"Census Tract 1048.10, Los Angeles, CA",5631.0,588.0,3947.0,691.0,12.0,19.0,32.0,46.0,...,17.0,1509.0,658.0,84.0,78.0,53.0,58.0,31.0,55.0,"MULTIPOLYGON (((-118.41938 34.25224, -118.4177..."


In [12]:
# trim the data to the bare minimum columns
tracts = tracts[['geoid', 'name', 'geometry']]

In [13]:
tracts.sample()

,geoid,name,geometry
487,14000US06037199201,"Census Tract 1992.01, Los Angeles, CA","MULTIPOLYGON (((-118.21570 34.07641, -118.2156..."


In [14]:
# drop the beginning of the geoid to make it match the FIPS column in the stats sheet
tracts['geoid'] = tracts['geoid'].str.replace('14000US','')
tracts.tail()

,geoid,name,geometry
2342,06037980033,"Census Tract 9800.33, Los Angeles, CA","MULTIPOLYGON (((-118.24897 33.75590, -118.2470..."
2343,06037990100,"Census Tract 9901, Los Angeles, CA","MULTIPOLYGON (((-118.95114 33.99643, -118.9505..."
2344,06037990200,"Census Tract 9902, Los Angeles, CA","MULTIPOLYGON (((-118.63598 34.03255, -118.6325..."
2345,06037990300,"Census Tract 9903, Los Angeles, CA","MULTIPOLYGON (((-118.47656 33.75038, -118.4661..."
2346,16000US0644000,"Los Angeles, CA","MULTIPOLYGON (((-118.66818 34.18987, -118.6681..."


In [15]:
tracts=tracts.drop(2346)


In [16]:
tracts.columns = ['FIPS', 'name', 'geometry']

In [17]:
tracts.sample()

,FIPS,name,geometry
1411,06037480903,"Census Tract 4809.03, Los Angeles, CA","MULTIPOLYGON (((-118.14451 34.07127, -118.1445..."


In [18]:
tracts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2346 entries, 0 to 2345
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   FIPS      2346 non-null   object  
 1   name      2346 non-null   object  
 2   geometry  2346 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 73.3+ KB


In [19]:
# converting the FIPS from an object to an int64
tracts.FIPS=tracts.FIPS.astype(int)

In [20]:
# checking
tracts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2346 entries, 0 to 2345
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   FIPS      2346 non-null   int64   
 1   name      2346 non-null   object  
 2   geometry  2346 non-null   geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 73.3+ KB


In [21]:
# now to bring in race data
stats = pd.read_csv('acs2019 five year stats.csv')

In [22]:
stats.sample(5)

,FIPS,Area Name,Census Tract,Total Population:,Total: White,Total: Black,Total: AmerIndian,Total: Asian,Total: Hawaiian and PacIslander,Total: Other,...,% Total: Black,% Total: AmerIndian,% Total: Asian,% Total: Hawaiian and PacIslander,% Total: Other,% Total: Mixed,Total: Not Hispanic or Latino,Total: Hispanic or Latino,% Total: Not Hispanic or Latino,% Total: Hispanic or Latino
787,6037239320,Census Tract 2393.20,239320,4086,1202,440,147,31,0,2093,...,10.77,3.6,0.76,0.00,51.22,4.23,635,3451,15.54,84.46
578,6037211121,Census Tract 2111.21,211121,2253,759,158,0,877,0,343,...,7.01,0.0,38.93,0.00,15.22,5.15,1461,792,64.85,35.15
1316,6037433102,Census Tract 4331.02,433102,5348,1125,0,0,2241,60,1922,...,0.00,0.0,41.90,1.12,35.94,0.00,2316,3032,43.31,56.69
2187,6037900701,Census Tract 9007.01,900701,5012,2424,1507,25,133,14,716,...,30.07,0.5,2.65,0.28,14.29,3.85,3194,1818,63.73,36.27
1067,6037400404,Census Tract 4004.04,400404,4268,3339,0,0,576,2,143,...,0.00,0.0,13.50,0.05,3.35,4.87,3447,821,80.76,19.24


In [23]:
# making sure out FIPS codes match types
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   FIPS                               2346 non-null   int64  
 1   Area Name                          2346 non-null   object 
 2   Census Tract                       2346 non-null   int64  
 3   Total Population:                  2346 non-null   int64  
 4   Total: White                       2346 non-null   int64  
 5   Total: Black                       2346 non-null   int64  
 6   Total: AmerIndian                  2346 non-null   int64  
 7   Total: Asian                       2346 non-null   int64  
 8   Total: Hawaiian and PacIslander    2346 non-null   int64  
 9   Total: Other                       2346 non-null   int64  
 10  Total: Mixed                       2346 non-null   int64  
 11  % Total: White                     2327 non-null   float

In [24]:
# check the size of each dataset
print(tracts.shape)
print(stats.shape)

(2346, 3)
(2346, 22)


In [25]:
# merge the datasets
merged = tracts.merge(stats, # left dataset = shapes, right dataset = district ID's
                                                    on='FIPS', # the column to merge with
                                                    how='left', # use a left outer join, which means all of the entries in expulsions will be preserved
                                                    indicator=True) # create a new column '_merge' that tells us how the merge went


In [26]:
# check the size of each dataset
print(merged.shape)

(2346, 25)


In [27]:
merged.sample(7)

,FIPS,name,geometry,Area Name,Census Tract,Total Population:,Total: White,Total: Black,Total: AmerIndian,Total: Asian,...,% Total: AmerIndian,% Total: Asian,% Total: Hawaiian and PacIslander,% Total: Other,% Total: Mixed,Total: Not Hispanic or Latino,Total: Hispanic or Latino,% Total: Not Hispanic or Latino,% Total: Hispanic or Latino,_merge
1498,6037503104,"Census Tract 5031.04, Los Angeles, CA","MULTIPOLYGON (((-118.04667 33.91768, -118.0466...",Census Tract 5031.04,503104,2542,796,5,49,88,...,1.93,3.46,6.02,56.77,0.31,375,2167,14.75,85.25,both
1696,6037542900,"Census Tract 5429, Los Angeles, CA","MULTIPOLYGON (((-118.25181 33.89583, -118.2489...",Census Tract 5429,542900,3254,1238,431,28,13,...,0.86,0.40,0.34,45.61,1.51,495,2759,15.21,84.79,both
747,6037234200,"Census Tract 2342, Los Angeles, CA","MULTIPOLYGON (((-118.33510 34.02143, -118.3351...",Census Tract 2342,234200,2424,401,1694,0,184,...,0.00,7.59,1.24,1.36,3.38,2261,163,93.28,6.72,both
930,6037276604,"Census Tract 2766.04, Los Angeles, CA","MULTIPOLYGON (((-118.43530 33.95967, -118.4352...",Census Tract 2766.04,276604,4624,2620,371,87,917,...,1.88,19.83,0.00,7.81,5.80,3863,761,83.54,16.46,both
1651,6037536104,"Census Tract 5361.04, Los Angeles, CA","MULTIPOLYGON (((-118.18393 33.95554, -118.1838...",Census Tract 5361.04,536104,3900,2669,59,19,31,...,0.49,0.79,0.00,27.36,1.41,244,3656,6.26,93.74,both
1210,6037407101,"Census Tract 4071.01, Los Angeles, CA","MULTIPOLYGON (((-117.98110 34.05122, -117.9805...",Census Tract 4071.01,407101,4883,2412,34,96,403,...,1.97,8.25,0.00,39.34,0.35,631,4252,12.92,87.08,both
1478,6037501900,"Census Tract 5019, Los Angeles, CA","MULTIPOLYGON (((-118.03780 33.95820, -118.0371...",Census Tract 5019,501900,4153,2222,89,67,68,...,1.61,1.64,0.00,35.85,5.25,1012,3141,24.37,75.63,both


In [28]:
merged.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [29]:
merged.set_crs('EPSG:3857', allow_override=True, inplace=True)

,FIPS,name,geometry,Area Name,Census Tract,Total Population:,Total: White,Total: Black,Total: AmerIndian,Total: Asian,...,% Total: AmerIndian,% Total: Asian,% Total: Hawaiian and PacIslander,% Total: Other,% Total: Mixed,Total: Not Hispanic or Latino,Total: Hispanic or Latino,% Total: Not Hispanic or Latino,% Total: Hispanic or Latino,_merge
0,6037101110,"Census Tract 1011.10, Los Angeles, CA","MULTIPOLYGON (((-118.30229 34.25870, -118.3009...",Census Tract 1011.10,101110,4283,3347,19,3,331,...,0.07,7.73,0.07,9.13,4.41,3095,1188,72.26,27.74,both
1,6037101122,"Census Tract 1011.22, Los Angeles, CA","MULTIPOLYGON (((-118.30334 34.27371, -118.3033...",Census Tract 1011.22,101122,3405,2981,74,15,220,...,0.44,6.46,0.00,1.41,1.97,3258,147,95.68,4.32,both
2,6037101210,"Census Tract 1012.10, Los Angeles, CA","MULTIPOLYGON (((-118.29945 34.25598, -118.2979...",Census Tract 1012.10,101210,6347,4647,198,0,486,...,0.00,7.66,0.00,14.51,1.50,3731,2616,58.78,41.22,both
3,6037101220,"Census Tract 1012.20, Los Angeles, CA","MULTIPOLYGON (((-118.28593 34.25227, -118.2859...",Census Tract 1012.20,101220,3702,2634,87,0,341,...,0.00,9.21,0.00,13.43,3.86,2338,1364,63.16,36.84,both
4,6037101300,"Census Tract 1013, Los Angeles, CA","MULTIPOLYGON (((-118.27822 34.25068, -118.2782...",Census Tract 1013,101300,3884,3350,205,0,194,...,0.00,4.99,0.00,1.16,2.32,3587,297,92.35,7.65,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,6037980031,"Census Tract 9800.31, Los Angeles, CA","MULTIPOLYGON (((-118.29105 33.75378, -118.2905...",Census Tract 9800.31,980031,1175,636,246,37,50,...,3.15,4.26,2.04,8.34,7.15,788,387,67.06,32.94,both
2342,6037980033,"Census Tract 9800.33, Los Angeles, CA","MULTIPOLYGON (((-118.24897 33.75590, -118.2470...",Census Tract 9800.33,980033,16,0,0,16,0,...,100.00,0.00,0.00,0.00,0.00,16,0,100.00,0.00,both
2343,6037990100,"Census Tract 9901, Los Angeles, CA","MULTIPOLYGON (((-118.95114 33.99643, -118.9505...",Census Tract 9901,990100,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,both
2344,6037990200,"Census Tract 9902, Los Angeles, CA","MULTIPOLYGON (((-118.63598 34.03255, -118.6325...",Census Tract 9902,990200,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,both


In [30]:
merged.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [31]:
# Do the spatial join

join = gpd.sjoin(wellsIDLE, merged, how='left', op='within')
join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3617 entries, 2 to 24232
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   LeaseName                          3617 non-null   object  
 1   WellNumber                         3617 non-null   object  
 2   WellStatus                         3617 non-null   object  
 3   WellType                           3617 non-null   object  
 4   WellTypeLa                         3617 non-null   object  
 5   OperatorCo                         3617 non-null   object  
 6   OperatorNa                         3617 non-null   object  
 7   Latitude                           3617 non-null   float64 
 8   Longitude                          3617 non-null   float64 
 9   geometry                           3617 non-null   geometry
 10  index_right                        0 non-null      float64 
 11  FIPS                              

In [32]:
merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2346 entries, 0 to 2345
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   FIPS                               2346 non-null   int64   
 1   name                               2346 non-null   object  
 2   geometry                           2346 non-null   geometry
 3   Area Name                          2346 non-null   object  
 4   Census Tract                       2346 non-null   int64   
 5   Total Population:                  2346 non-null   int64   
 6   Total: White                       2346 non-null   int64   
 7   Total: Black                       2346 non-null   int64   
 8   Total: AmerIndian                  2346 non-null   int64   
 9   Total: Asian                       2346 non-null   int64   
 10  Total: Hawaiian and PacIslander    2346 non-null   int64   
 11  Total: Other                       